# MCP Fundamentals

**Extending Strands Agents with the Model Context Protocol**

---

Welcome to the world of **Model Context Protocol (MCP)**! This notebook introduces you to MCP, an open protocol that enables AI agents to connect with external tools and services. By the end of this 10-minute tutorial, you'll understand MCP basics and have your first MCP-powered agent running.

### 🎯 What You'll Learn

In this tutorial, you will:
- Understand what MCP is and why it's powerful
- Build a simple calculator MCP server
- Connect to an MCP server using Strands
- Use MCP tools with your agents
- Handle context managers properly
- Troubleshoot common MCP issues

### 🤔 What is MCP?

The **Model Context Protocol** is a standardized way for AI applications to:
- **Connect** to external services
- **Discover** available tools
- **Execute** operations safely
- **Receive** structured responses

Think of it as a universal adapter for AI agents!

## 📦 Step 1: Installing MCP Dependencies

### Required Packages
MCP support is built into Strands, but we need the MCP package for creating servers.

### 📚 What We're Installing
- **mcp**: The Model Context Protocol package
- **strands-agents**: Already installed (includes MCP client support)

In [ ]:
# Install MCP package
%pip install mcp -q

# Verify installation
import mcp
from mcp.server import FastMCP
from mcp import stdio_client, StdioServerParameters

print("✅ MCP package installed successfully!")
print("   Ready to build MCP servers and connect agents! 🚀")

## 🔧 Step 2: Building Your First MCP Server

### Inline Calculator Server
Let's create a simple calculator MCP server right here in the notebook. This demonstrates the core concepts without external files.

### 🎯 Key Concepts
- **FastMCP**: Simple framework for building MCP servers
- **@mcp.tool**: Decorator that exposes functions as tools
- **Type hints**: Required for proper tool discovery

In [ ]:
from mcp.server import FastMCP

# Create a simple calculator MCP server
calc_mcp = FastMCP("Simple Calculator")

@calc_mcp.tool(description="Add two numbers together")
def add(x: float, y: float) -> float:
    """Add two numbers and return the result."""
    return x + y

@calc_mcp.tool(description="Subtract the second number from the first")
def subtract(x: float, y: float) -> float:
    """Subtract y from x and return the result."""
    return x - y

@calc_mcp.tool(description="Multiply two numbers together")
def multiply(x: float, y: float) -> float:
    """Multiply two numbers and return the result."""
    return x * y

@calc_mcp.tool(description="Divide the first number by the second")
def divide(x: float, y: float) -> float:
    """Divide x by y and return the result."""
    if y == 0:
        raise ValueError("Cannot divide by zero")
    return x / y

print("🧮 Calculator MCP server defined!")
print("   Tools: add, subtract, multiply, divide")

## 🔌 Step 3: Understanding MCP Transports

### Transport Types
MCP supports different ways to communicate between clients and servers:

| Transport | Use Case | Example |
|-----------|----------|----------|
| **stdio** | Local command-line tools | Python scripts, Node.js apps |
| **SSE** | HTTP with Server-Sent Events | Web services |
| **HTTP** | Standard HTTP requests | REST APIs |

### 📡 We'll Use stdio
For this tutorial, we'll use stdio (standard input/output) as it's the simplest and works locally.

## 🤖 Step 4: Connecting Strands to MCP

### The MCPClient
Strands provides `MCPClient` to connect to MCP servers. Here's how it works:

1. **Create a client** with a transport configuration
2. **Use context manager** to maintain the connection
3. **Discover tools** from the server
4. **Create agent** with the tools

### ⚠️ Important: Context Managers
Always use `with` statements when working with MCP!

In [ ]:
from strands import Agent
from strands.tools.mcp import MCPClient
from strands.models import BedrockModel
import boto3

# Configure AWS Bedrock (or use any other model)
session = boto3.Session(profile_name='default')
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    boto_session=session
)

print("✅ Model configured successfully!")
print("   Using Claude 3.5 Sonnet via AWS Bedrock")

## 🚀 Step 5: Using External MCP Server

### Running the Calculator Server
For real-world usage, MCP servers run as separate processes. Let's connect to our calculator server file.

### 📝 Platform-Specific Commands
The command syntax differs between Windows and macOS/Linux.

In [ ]:
import platform
from mcp import stdio_client, StdioServerParameters

# Detect platform for proper command syntax
is_windows = platform.system() == "Windows"

# Create MCP client for our calculator server
# The server file is in src/mcp_servers/calculator_server.py
stdio_mcp_client = MCPClient(lambda: stdio_client(
    StdioServerParameters(
        command="python",
        args=["../src/mcp_servers/calculator_server.py"]
    )
))

print("🔌 MCP client configured!")
print(f"   Platform: {platform.system()}")
print("   Server: Calculator MCP Server")

## 💬 Step 6: Creating an MCP-Powered Agent

### Context Manager Magic
Watch how we use the `with` statement to keep the MCP connection alive while the agent works.

### 🎯 The Pattern
```python
with mcp_client:
    # Connection is open here
    tools = mcp_client.list_tools_sync()
    agent = Agent(tools=tools)
    response = agent("your prompt")
# Connection closes automatically
```

In [ ]:
# Create an agent with MCP tools
print("🤖 Creating MCP-powered agent...\n")

# Use context manager to maintain MCP connection
with stdio_mcp_client:
    # List available tools from the MCP server
    tools = stdio_mcp_client.list_tools_sync()
    
    print("📋 Available tools:")
    for tool in tools:
        print(f"   - {tool.name}: {tool.description}")
    
    # Create agent with MCP tools
    calculator_agent = Agent(
        model=bedrock_model,
        system_prompt="""You are a helpful calculator assistant. 
        Use the available math tools to solve problems step by step.
        Always show your work and explain your calculations.""",
        tools=tools
    )
    
    print("\n✅ Agent created with MCP tools!")

## 🧮 Step 7: Testing Your MCP Agent

### Real Calculations
Let's test our calculator agent with various math problems to see MCP in action!

In [ ]:
# Test 1: Simple calculation
with stdio_mcp_client:
    tools = stdio_mcp_client.list_tools_sync()
    agent = Agent(model=bedrock_model, tools=tools)
    
    print("🔍 Test 1: Simple Calculation")
    print("="*50)
    response = agent("What is 25 + 17?")
    print(f"🤖 Agent: {response}")
    print("\n" + "="*50 + "\n")

In [ ]:
# Test 2: Multi-step calculation
with stdio_mcp_client:
    tools = stdio_mcp_client.list_tools_sync()
    agent = Agent(model=bedrock_model, tools=tools)
    
    print("🔍 Test 2: Multi-Step Calculation")
    print("="*50)
    response = agent("Calculate (10 + 5) * 3 - 8")
    print(f"🤖 Agent: {response}")
    print("\n" + "="*50 + "\n")

In [ ]:
# Test 3: Error handling
with stdio_mcp_client:
    tools = stdio_mcp_client.list_tools_sync()
    agent = Agent(model=bedrock_model, tools=tools)
    
    print("🔍 Test 3: Error Handling")
    print("="*50)
    response = agent("What happens if I divide 10 by 0?")
    print(f"🤖 Agent: {response}")
    print("\n" + "="*50 + "\n")

## ⚠️ Step 8: Common Pitfalls & Solutions

### The #1 MCP Mistake
Using an agent outside the context manager:

```python
# ❌ WRONG - This will fail!
with mcp_client:
    agent = Agent(tools=mcp_client.list_tools_sync())
response = agent("Calculate 2+2")  # MCPClientInitializationError!

# ✅ CORRECT - Keep everything inside
with mcp_client:
    agent = Agent(tools=mcp_client.list_tools_sync())
    response = agent("Calculate 2+2")  # Works!
```

### 🔧 Debugging Tips
1. **Check server is running**: Ensure your MCP server starts correctly
2. **Verify paths**: Use absolute paths if relative paths fail
3. **Platform differences**: Windows vs macOS/Linux commands
4. **Context manager**: Always use `with` statements

## 🌐 Step 9: Other MCP Transports

### Beyond stdio
MCP supports multiple transport mechanisms. Here's a quick overview:

In [ ]:
# Example: SSE (Server-Sent Events) transport
from mcp.client.sse import sse_client

# This would connect to an HTTP-based MCP server
# sse_mcp_client = MCPClient(lambda: sse_client("http://localhost:8000/sse"))

# Example: Streamable HTTP transport
from mcp.client.streamable_http import streamablehttp_client

# This would connect to a different HTTP endpoint
# http_mcp_client = MCPClient(lambda: streamablehttp_client("http://localhost:8000/mcp"))

print("📡 MCP Transport Options:")
print("   - stdio: Local command-line tools (what we used)")
print("   - SSE: Server-Sent Events over HTTP")
print("   - HTTP: Streamable HTTP transport")
print("   - Custom: Implement your own transport!")

## 🎓 Step 10: Best Practices

### MCP Development Guidelines
Here are the key principles for working with MCP:

In [ ]:
print("🎓 MCP BEST PRACTICES")
print("=" * 60)

best_practices = {
    "🔧 Tool Design": [
        "Use clear, descriptive tool names",
        "Include comprehensive descriptions",
        "Add type hints for all parameters",
        "Handle errors gracefully"
    ],
    "🔌 Connection Management": [
        "Always use context managers (with statements)",
        "Initialize resources in server startup",
        "Clean up resources on shutdown",
        "Handle connection failures gracefully"
    ],
    "🚀 Performance": [
        "Keep tools focused and fast",
        "Avoid blocking operations",
        "Cache expensive computations",
        "Use appropriate timeout values"
    ],
    "🔒 Security": [
        "Validate all inputs",
        "Limit resource access",
        "Use authentication for network servers",
        "Log security-relevant events"
    ]
}

for category, tips in best_practices.items():
    print(f"\n{category}")
    for tip in tips:
        print(f"   • {tip}")

## 🎉 Congratulations!

### 🏆 What You've Accomplished

In just 10 minutes, you've:
- ✅ Understood the Model Context Protocol
- ✅ Built a calculator MCP server
- ✅ Connected Strands agents to MCP
- ✅ Used MCP tools in practice
- ✅ Learned about context managers
- ✅ Explored different transports

### 🚀 What's Next?

Now that you understand MCP basics, you're ready to:
1. **Build Complex Servers** - Database access, API integrations
2. **Use Multiple Servers** - Combine tools from different sources
3. **Deploy MCP Servers** - Run servers in production
4. **Create Custom Transports** - Extend MCP capabilities

### 💡 Key Takeaways

1. **MCP = Universal Tool Protocol**: Connect any service to any AI
2. **Context Managers are Critical**: Always use `with` statements
3. **Tools Need Types**: Type hints enable proper discovery
4. **Start Simple**: stdio transport is perfect for learning

### 📚 Resources

- [MCP Documentation](https://modelcontextprotocol.io)
- [Strands MCP Guide](https://strandsagents.com/0.1.x/user-guide/concepts/tools/mcp-tools/)
- [FastMCP Documentation](https://github.com/anthropics/fastmcp)

### 🌟 Challenge Yourself

Try extending the calculator server with:
- Square root and power functions
- Memory storage (store/recall)
- Unit conversions
- Scientific calculator functions

Happy building with MCP! 🚀🤖✨